In [152]:
#-*- coding:utf8 -*- 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 同一个cell做多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 显示所有列
pd.set_option('display.max_columns', None)

## 一、数据探索
> a 由于没有给出字段含义，外加不熟悉数据背后的业务逻辑，只能根据字段名称猜测 和 数据内容进行简单的维度过滤    
> b 对于缺失数据，根据经验，缺失值的个数超过5% 该部分数据直接删除即可（只是电商行业，其他的不清楚）   

### 1、数据读取 与 简单探索
对数据集进行简单探索，发现

- 1、70种float64类型， 13种int64类型，7种object类型； 

- 2、根据经验 object的类型的字段较好判断，且一般与目标关联不大,该部分主要字段有：    
    ['trade_no', 'bank_card_no', 'reg_preference_for_trad', 'source', 'id_name', 'latest_query_time', 'loans_latest_time']    
    结合字段英文含义 并 观察该部分数据的主要内容，发现该部分主要为用户的属性信息，但是需要保留trade_no来区别用户（暂时还确定）


In [153]:
# 读取原始数据 所有的数据的改动使用副本数据
df = pd.read_csv('./data/data.csv', encoding='gb18030') # gbk系列编码 直接使用字符超集
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)  # 第一列修改列名
df.head()

# 不同数据类型的分布
df.dtypes.value_counts()

# 每种类型的变量名称
print('Object Type : ',list(df.select_dtypes(include=['object']).columns.values),'\n')
print('Int64 Type : ',list(df.select_dtypes(include=['int64']).columns.values),'\n')
print('Float64 Type : ',list(df.select_dtypes(include=['float64']).columns.values))

# object列内容
df.select_dtypes(include=['object']).head()

# 删除无效字段 确定第一步数据
df_type1 = df.drop(list(df.select_dtypes(include=['object']).columns)[1:], axis=1)

,id,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,student_feature,repayment_capability,is_high_user,number_of_trans_from_2011,first_transaction_time,historical_trans_amount,historical_trans_day,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,reg_preference_for_trad,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,cross_consume_count_last_1_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,status,source,first_transaction_day,trans_day_last_12_month,id_name,apply_score,apply_credibility,query_org_count,query_finance_count,query_cash_count,query_sum_count,latest_query_time,latest_one_month_apply,latest_three_month_apply,latest_six_month_apply,loans_score,loans_credibility_behavior,loans_count,loans_settle_count,loans_overdue_count,loans_org_count_behavior,consfin_org_count_behavior,loans_cash_count,latest_one_month_loan,latest_three_month_loan,latest_six_month_loan,history_suc_fee,history_fail_fee,latest_one_month_suc,latest_one_month_fail,loans_long_time,loans_latest_time,loans_credit_limit,loans_credibility_limit,loans_org_count_current,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,3.0,NaN,19890,0,30.0,20130817.0,149050,151.0,0.40,34030,7.0,3920,0.15,1020,0.55,一线城市,4.0,19.0,4.0,19.0,1.0,1.0,2.0,2.0,5.0,2170,6.0,0.0,1970,18040,0.0,1,xs,1738.0,85.0,蒋红,583.0,79.0,8.0,2.0,6.0,10.0,2018-04-25,2.0,5.0,8.0,552.0,73.0,37.0,34.0,2.0,10.0,1.0,9.0,1.0,1.0,13.0,37.0,7.0,1.0,0.0,341.0,2018-04-19,2200.0,72.0,9.0,10.0,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,4.0,1.0,16970,0,23.0,20160402.0,302910,224.0,0.35,10590,5.0,6950,0.05,1210,0.50,一线城市,13.0,30.0,13.0,30.0,0.0,0.0,3.0,3.0,330.0,2100,9.0,0.0,1820,15680,0.0,0,xs,779.0,84.0,崔向朝,653.0,73.0,7.0,4.0,2.0,8.0,2018-05-03,2.0,6.0,8.0,635.0,76.0,37.0,36.0,0.0,17.0,5.0,12.0,2.0,2.0,8.0,49.0,4.0,2.0,1.0,353.0,2018-05-05,2000.0,74.0,12.0,12.0,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,1.0,NaN,9710,0,9.0,20170617.0,11520,31.0,1.00,5710,5.0,840,0.65,570,0.65,一线城市,0.0,68.0,0.0,68.0,0.0,3.0,6.0,6.0,0.0,0,3.0,0.0,0,0,0.0,1,xs,338.0,95.0,王中云,654.0,76.0,11.0,5.0,5.0,16.0,2018-05-05,5.0,5.0,14.0,633.0,83.0,4.0,2.0,0.0,3.0,1.0,2.0,2.0,2.0,4.0,2.0,2.0,1.0,1.0,157.0,2018-05-01,1500.0,77.0,2.0,2.0,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,3.0,NaN,6210,0,33.0,20130516.0,491130,360.0,0.15,91690,7.0,46850,0.05,1290,0.45,三线城市,6.0,8.0,6.0,8.0,0.0,1.0,8.0,8.0,31700.0,8140,9.0,0.0,2700,27970,0.0,0,xs,1831.0,82.0,何洋洋,595.0,79.0,12.0,7.0,4.0,22.0,2018-05-05,3.0,16.0,17.0,542.0,75.0,85.0,81.0,4.0,22.0,5.0,17.0,2.0,4.0,34.0,91.0,26.0,2.0,0.0,355.0,2018-05-03,1800.0,74.0,17.0,18.0,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,1.0,NaN,11

float64    70
int64      13
object      7
dtype: int64

Object Type :  ['trade_no', 'bank_card_no', 'reg_preference_for_trad', 'source', 'id_name', 'latest_query_time', 'loans_latest_time'] 

Int64 Type :  ['id', 'custid', 'take_amount_in_later_12_month_highest', 'repayment_capability', 'is_high_user', 'historical_trans_amount', 'trans_amount_3_month', 'abs', 'avg_price_last_12_month', 'max_cumulative_consume_later_1_month', 'pawns_auctions_trusts_consume_last_1_month', 'pawns_auctions_trusts_consume_last_6_month', 'status'] 

Float64 Type :  ['low_volume_percent', 'middle_volume_percent', 'trans_amount_increase_rate_lately', 'trans_activity_month', 'trans_activity_day', 'transd_mcc', 'trans_days_interval_filter', 'trans_days_interval', 'regional_mobility', 'student_feature', 'number_of_trans_from_2011', 'first_transaction_time', 'historical_trans_day', 'rank_trad_1_month', 'avg_consume_less_12_valid_month', 'top_trans_count_last_1_month', 'avg_price_top_last_12_valid_month', 'trans_top_time_last_1_month', 'trans_top_time_last_6_month', 'co

,trade_no,bank_card_no,reg_preference_for_trad,source,id_name,latest_query_time,loans_latest_time
0,20180507115231274000000023057383,卡号1,一线城市,xs,蒋红,2018-04-25,2018-04-19
1,20180507121002192000000023073000,卡号1,一线城市,xs,崔向朝,2018-05-03,2018-05-05
2,20180507125159718000000023114911,卡号1,一线城市,xs,王中云,2018-05-05,2018-05-01
3,20180507121358683000000388283484,卡号1,三线城市,xs,何洋洋,2018-05-05,2018-05-03
4,20180507115448545000000388205844,卡号1,一线城市,xs,赵洋,2018-04-15,2018-01-07


### 2 缺失数据处理（数值型字段）
- 1、将缺失值比率阈值设置为5%（最好根据行业标准制定或者专家制定），超过该值得将删除该字段；
- 2、缺失值得补充：因不了解该部分业务，使用平均值进行填补
- 3、最好不要直接删除空缺值字段


In [154]:
# 删除缺失值比率超过5%的字段
isNuRatio = 0.05 # 缺失率 5%
res1 = df_type1.isnull().sum()/df_type1.shape[0]
df_type2 = df_type1.drop(res1[res1 >= isNuRatio].index.values ,axis=1) 

df_type2.head()

# 缺失值补充  以该字段的平均值进行填补
col_list = df_type2.isnull().sum()[df_type2.isnull().sum()>0].index.values
for col in col_list:
    ser1 = df_type2[col]
    df_type2[col] = ser1.fillna(value=ser1.dropna().mean())

df_type2.isnull().sum()[df_type2.isnull().sum()>0] # 缺失值已经补全          

df_base = df_type2.copy()

# 数据类型转换 int64 转换为 np.float64
for col in df_base.select_dtypes(include=['int64']).columns.values:
    df_base[col] = df_base[col].astype(np.float64)

,id,custid,trade_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,repayment_capability,is_high_user,number_of_trans_from_2011,first_transaction_time,historical_trans_amount,historical_trans_day,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,status,first_transaction_day,trans_day_last_12_month
0,5,2791858,20180507115231274000000023057383,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,3.0,19890,0,30.0,20130817.0,149050,151.0,0.40,34030,7.0,3920,0.15,1020,0.55,4.0,19.0,4.0,19.0,1.0,2.0,2.0,5.0,2170,6.0,0.0,1970,18040,0.0,1,1738.0,85.0
1,10,534047,20180507121002192000000023073000,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,4.0,16970,0,23.0,20160402.0,302910,224.0,0.35,10590,5.0,6950,0.05,1210,0.50,13.0,30.0,13.0,30.0,0.0,3.0,3.0,330.0,2100,9.0,0.0,1820,15680,0.0,0,779.0,84.0
2,12,2849787,20180507125159718000000023114911,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,1.0,9710,0,9.0,20170617.0,11520,31.0,1.00,5710,5.0,840,0.65,570,0.65,0.0,68.0,0.0,68.0,3.0,6.0,6.0,0.0,0,3.0,0.0,0,0,0.0,1,338.0,95.0
3,13,1809708,20180507121358683000000388283484,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,3.0,6210,0,33.0,20130516.0,491130,360.0,0.15,91690,7.0,46850,0.05,1290,0.45,6.0,8.0,6.0,8.0,1.0,8.0,8.0,31700.0,8140,9.0,0.0,2700,27970,0.0,0,1831.0,82.0
4,14,2499829,20180507115448545000000388205844,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,1.0,11150,0,12.0,20170312.0,61470,63.0,0.65,9770,6.0,760,1.00,1110,0.50,0.0,66.0,0.0,66.0,3.0,3.0,3.0,0.0,1000,3.0,0.0,0,6410,0.0,1,435.0,88.0


Series([], dtype: int64)

### 二、数据维度处理
> 由于不了解业务，无法从业务角度给出更好的数据维度组合

经上述处理后， 仍然包含43个维度：   
1、去除类别信息，简单使用PCA进行简单的降维；    
2、因是类别预测，降维之后，在根据相关系数进行维度规约（该部分暂缓）；


In [155]:
# 数据分列
X = df_base.drop(['status','trade_no'],axis=1)
y = df_base['status']
trade_no = df_base['status']


In [156]:
## 使用PCA降至5维
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(X)
ratio_array = pca.explained_variance_ratio_

ratio_array.sum()  # 方差占比99%以上 不过这种降维有些粗暴了 应该尝试多种降维方法

X_new = pca.transform(X)

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

0.9991473055379295

## 三、选择模型 
> 时间关系，直接选择LR模型进行建模预测

- 1、首先进行数据归一化
- 2、LR建模 （时间关系 没做RepeatedKFold）

In [157]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection  import train_test_split

# 数据切分
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=2019)

# 归一化处理
S=StandardScaler(); # 数据类型未处理
S.fit(X_train)
X_train_stand = S.transform(X_train)
X_test_stand = S.transform(X_test)

# 模型搭建和预测
Log = LogisticRegression(C=10)
Log.fit(X_train_stand, y_train)  #训练模型
prediction = Log.predict(X_test_stand)  #用训练的模型Log来预测测试数据

# 时间关系 只验证准确率
from sklearn.metrics import accuracy_score
y_pred = [0,2,1,3]
y_true = [0,1,2,3]
accuracy_score(y_test, prediction) # 75%准确率 真低~~~

StandardScaler(copy=True, with_mean=True, with_std=True)

D:\Anaconda3.4_setup\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

0.7526278906797477